In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00


In [3]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=5ebd5652c20956e0bfd79b2ffed81effc98ab480bc1bfc2dca6456f800d2a917
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [1]:


import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
import sacremoses
import sentencepiece


In [2]:
# Load your parallel corpus data into a pandas DataFrame
data = pd.read_csv("data_without_hadith.csv", index_col=0)  # Adapt the file name and structure

# Define your fine-tuning dataset class
class TranslationDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data.iloc[idx]["English"]
        target_text = self.data.iloc[idx]["Arabic"]

        # Tokenize source and target sentences
        source_tokens = self.tokenizer.encode(source_text, padding="max_length", truncation=True, return_tensors="pt")

        with self.tokenizer.as_target_tokenizer():
          target_tokens = self.tokenizer.encode(target_text, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "input_ids": source_tokens.squeeze(),
            "attention_mask": source_tokens.squeeze().gt(0),  # Create attention mask
            "labels": target_tokens.squeeze(),
        }

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data

,Arabic,English
0,تفريج الكروب فى تدبير الحروب,TAFRIJ AL-KURUB FI TADBIR AL-HURUBA Muslim Man...
1,مقدمة,INTRODUCTION
2,بسم الله الرحمن الرحيم.. مؤيد الإسلام من سلطان...,"IN THE NAME OF GOD, THE MERCIFUL, THE COMPASSI..."
3,ومسعد جده العالى بإبادة أعدائه الطغاة المارقين...,And [he is] the cause of his noble sire’s happ...
4,وأشهد أن لا إله إلا الله وحده لا شريك له، شهاد...,"I declare that there is no god but God alone, ..."
...,...,...
61728,وقال المأموني في تاريخه: وقيل إن أهل المدينة ع...,"Al-Mamuni says, in his History: “It is stated ..."
61729,ثم إن كل من بين مكة والمدينة ممن قرب من البصرة...,"Then, all who dwelt between Mecca and Medina, ..."
61730,قلت: وذكر كشاجم في كتاب المصايد والمطارد” أن ا...,"Kushajim says, in his work entitled Al-Masaid ..."
61731,حدثنا أبو بكر بن أبي شيبة، حدثنا أسود بن عامر،...,“I heard the Prophet(ﷺ) say: ‘One of the porte...


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Load pretrained model and tokenizer for Arabic to English translation
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained('/content/drive/MyDrive/first_model').to(device)

In [7]:

# Create fine-tuning dataset and dataloader
train_dataset = TranslationDataset(data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

num_epochs = 3
best_loss = float('inf')  # Initialize with a high value

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}] - Batch Loss: {loss.item():.4f}')

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Save the best model based on training loss
    if average_loss < best_loss:
        best_loss = average_loss
        model.save_pretrained("seconed_model")
        model.save_pretrained('/content/drive/My Drive/seconed_model')

print("Training finished.")

Epoch [1/3] - Batch Loss: 0.2335
Epoch [1/3] - Batch Loss: 0.2298
Epoch [1/3] - Batch Loss: 0.2736
Epoch [1/3] - Batch Loss: 0.1906
Epoch [1/3] - Batch Loss: 0.1655
Epoch [1/3] - Batch Loss: 0.1460
Epoch [1/3] - Batch Loss: 0.2521
Epoch [1/3] - Batch Loss: 0.2256
Epoch [1/3] - Batch Loss: 0.1377
Epoch [1/3] - Batch Loss: 0.2975
Epoch [1/3] - Batch Loss: 0.1858
Epoch [1/3] - Batch Loss: 0.2505
Epoch [1/3] - Batch Loss: 0.2289
Epoch [1/3] - Batch Loss: 0.2480
Epoch [1/3] - Batch Loss: 0.2809
Epoch [1/3] - Batch Loss: 0.1861
Epoch [1/3] - Batch Loss: 0.1552
Epoch [1/3] - Batch Loss: 0.2442
Epoch [1/3] - Batch Loss: 0.3036
Epoch [1/3] - Batch Loss: 0.2207
Epoch [1/3] - Batch Loss: 0.1262
Epoch [1/3] - Batch Loss: 0.2442
Epoch [1/3] - Batch Loss: 0.2573
Epoch [1/3] - Batch Loss: 0.2408
Epoch [1/3] - Batch Loss: 0.3372
Epoch [1/3] - Batch Loss: 0.3412
Epoch [1/3] - Batch Loss: 0.1750
Epoch [1/3] - Batch Loss: 0.3841
Epoch [1/3] - Batch Loss: 0.3510
Epoch [1/3] - Batch Loss: 0.1026
Epoch [1/3

In [9]:
stock_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ar').to(device)

In [10]:
hunain_model = MarianMTModel.from_pretrained('/content/drive/MyDrive/first_model').to(device)

In [11]:
# Inference function
def translate_english_to_arabic_stock(input_text):
    input_text = [input_text]
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    translated_ids = stock_model.generate(input_ids, max_length=len(str(input_text))+10,num_beams=100).to(device)
    with tokenizer.as_target_tokenizer():
      translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

In [12]:
# Inference function
def translate_english_to_arabic_hunain(input_text):
    input_text = [input_text]
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    translated_ids = model.generate(input_ids, max_length=len(str(input_text))+10,num_beams=100).to(device)
    with tokenizer.as_target_tokenizer():
      translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

In [13]:

def split_text_into_lines(text, max_words_per_line=100):
    words = text.split()
    lines = []
    current_line = []

    for word in words:
        if len(' '.join(current_line + [word])) <= max_words_per_line:
            current_line.append(word)
        else:
            lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        lines.append(' '.join(current_line))

    return lines




In [14]:

# Example usage
input_text =  "The concept of beauty is multifaceted and often subjective."
translated_text = translate_english_to_arabic_stock(input_text)

input_lines = split_text_into_lines(input_text)
translated_lines = split_text_into_lines(translated_text)

print("Input:")
for line in input_lines:
    print(line)

print("\nTranslated:")
for line in translated_lines:
    print(u'{}'.format(line))


Input:
The concept of beauty is multifaceted and often subjective.

Translated:
ومفهوم الجمال هو مفهوم متعدد الأوجه وذوي في كثير من الأحيان.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [15]:

# Example usage
input_text =  "The concept of beauty is multifaceted and often subjective."
translated_text = translate_english_to_arabic_hunain(input_text)

input_lines = split_text_into_lines(input_text)
translated_lines = split_text_into_lines(translated_text)
print("Input:")
for line in input_lines:
    print(line)

print("\nTranslated:")
for line in translated_lines:
    print(u'{}'.format(line))

Input:
The concept of beauty is multifaceted and often subjective.

Translated:
ومفهوم الحسن هو المعاني المتعددة والذاتية في كثير من الأحيان.


In [ ]:


# Save your model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
